In [186]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
url ="https://finance.naver.com/sise/sise_market_sum.naver"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36' 

def get_stock_list():
    # 요청 
    response = requests.get(url, headers = {"User-Agent":user_agent})
    # 상태 코드 200 확인
    if response.status_code == 200: # 정상 응답 확인하기
        print("정상 상태 코드 확인", response.status_code)
        # 요청 받아온거 뷰티풀 소프에 저장 -> 파싱하기 위해
        soup = BeautifulSoup(response.text, 'lxml')
        tag_list_menu=soup.select("#contentarea > div.box_type_l > table.type_2 > thead > th.col")
        tag_list_com =soup.select("#contentarea > div.box_type_l > table.type_2 > tbody> tr a")
        tag_list = soup.select("#contentarea > div.box_type_l > table.type_2 > tbody > tr> td.number")
        # 긁어오기 
        print(tag_list_menu)
        # print(tag_list_com)
        stock_menu=[] # 메뉴
        stock_list=[] # 값
        stock_name=[] # 이름
        for menu in tag_list_menu:
            stock_menu.append(menu.get_text())
        print(stock_menu)
        for comp in tag_list_com:
            stock_name.append(comp.get_text().strip())
        for i in stock_name:
            if i == "":
                stock_name.remove(i)

        print(stock_name)
        for tag in tag_list:
            stock_list.append(tag.get_text().strip())
        print(stock_list)


In [187]:
get_stock_list()

정상 상태 코드 확인 200
[]
[]
['삼성전자', 'LG에너지솔루션', 'SK하이닉스', 'LG화학', '삼성바이오로직스', '삼성SDI', '삼성전자우', '현대차', 'POSCO홀딩스', '기아', 'NAVER', '포스코퓨처엠', '카카오', '셀트리온', '현대모비스', '삼성물산', 'KB금융', 'LG전자', '신한지주', 'SK이노베이션', 'LG', '삼성생명', 'SK', '하나금융지주', '두산에너빌리티', '한국전력', 'KT&G', '카카오뱅크', '삼성전기', '고려아연', '하이브', 'LG생활건강', 'HMM', 'SK텔레콤', '삼성화재', '한화솔루션', 'HD현대중공업', '크래프톤', '삼성에스디에스', 'S-Oil', '대한항공', '우리금융지주', '엔씨소프트', '아모레퍼시픽', '기업은행', 'KT', '카카오페이', '롯데케미칼', '메리츠금융지주', '현대글로비스']
['65,500', '200', '+0.31%', '100', '3,910,208', '5,969,783', '51.52', '12,720,083', '8.13', '17.07', '592,000', '1,000', '+0.17%', '500', '1,385,280', '234,000', '5.40', '213,015', '179.12', '5.75', '87,900', '600', '-0.68%', '5,000', '639,914', '728,002', '49.54', '1,421,710', '28.70', '3.56', '819,000', '14,000', '+1.74%', '5,000', '578,151', '70,592', '48.44', '322,709', '34.74', '6.95', '791,000', '21,000', '-2.59%', '2,500', '562,986', '71,174', '10.71', '72,756', '69.32', '11.42', '766,000', '11,000', '+1.46%', '5,000', '526,

In [192]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import re
"url ="https://finance.naver.com/sise/sise_market_sum.naver"
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36' "

def get_stock_list():
    # 요청 
    response = requests.get(url, headers = {"User-Agent":user_agent})
    # 상태 코드 200 확인
    if response.status_code == 200: # 정상 응답 확인하기
        print("정상 상태 코드 확인", response.status_code)
        # 요청 받아온거 뷰티풀 소프에 저장 -> 파싱하기 위해
        soup = BeautifulSoup(response.text, 'lxml')
        tag_list = soup.select("#contentarea > div.box_type_l > table.type_2 > tbody > tr")
        stock_list= []
        for tag in tag_list:
            stock_list.append(tag.get_text().strip())
        print(stock_list)
        

In [4]:
# ##################################################################
# Naver 증권 -> 코스피 시가총액 순으로 조회
#    ROOT: URL: https://finance.naver.com/sise/sise_market_sum.naver
#    페이지 이동:     ?&page=2
#    총 41 페이지
​
# Table column 
# 1. NO
# 2. 종목명
# 3. 현재가
# 4. 전일비
# 5. 등락률
# 6. 액면가
# 7. 시가총액
# 8. 상장주식수
# 9. 외국인비율
# 10.거래량
# 11.PER
# 12.ROI
# 13.토론실 링크
​
# css selector: 
##  테이블: table.type_2 > tbody > tr
​
# 주의
## 전일비: 화살표이미지 값
### 화살표: ▲ - 상승 - alt 속성의 값이 상승
###        ▼ - 하락 - alt 속성의 값이 하락
###        변경이 없을 경우 image가 없음
###        ↑ ↓ 은 상한 하한가로 alt 속성이 없음 => image명을 구분. (ico_up02.gif, ico_down02.gif)
# ##################################################################


import requests
import os
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

def main():
    base_url ="https://finance.naver.com/sise/sise_market_sum.naver?&page={}"
    headers={ 
        'user_Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'

    }

    # 요청 
    result_list= [] # 전체 결과를 담을 리스트
    # page= 1
    for page in range(1,42):
        url = base_url.format(page)
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text,'lxml')
            tr_list= soup.select("table.type_2 > tbody > tr")
            # print(tr_list[2])
            for tr in tr_list:
                td_list = tr.find_all("td")
                if len(td_list) == 1: # 선을 그리는 용으로 사용된 tr
                    continue
                td_content_list=[] # 한행의 조회 결과를 담을(td들의 값)리스트
                for idx, td in enumerate(td_list):
                    txt= td.text.strip()
                    if idx == 3: # 전일비 <td>
                        img_tag = td.find("img")
                        if img_tag != None: # 앞에 이미지(상승,하락)가 있는 경우,
                            alt_attr = img_tag.get("alt")
                            if alt_attr == None: # <img>에 alt 속성이 없는 경우 -> 상한가/하한가
                                #<img>의 src 속성값을 조회
                                alt_attr = "상한 "if img_tag.get('src').endswith('ico_up02.jpg') else '하한'
                            txt =alt_attr + txt
                    elif idx == 12:
                        continue                                                   [ "asdasdasd", "bererefewf" ,"3ewrewrwerwer"]
                    td_content_list.append(txt) # 한 행을 구성하는 컬럼값을 저장   [ [a . ~~~], [b344231434234230493049],[334234234234234]]
                result_list.append(td_content_list) #한 행을 저장                      
                print(result_list)
                print(len(result_list))

            
        else:
            print(f"{page}실패")
            continue
    return result_list
if __name__ == "__name__":
    import time
    start = time.time()
    result= main()
    print(len(result))
    end = time.time()
    print("걸린시간:",end-start)
    save_path ="stock"
    os.makedirs(save_path, exist_ok=True)
    file_name= datetime.now().strftime("%Y-%m-%d_코피피종목별 시세정보.csv")

    df = pd.DataFrame(result)
    df.to_csv(f"{save_path}/{file_name}",index=False,columns=['No', '종목명', '현재가',	'전일비', '등락률','액면가','시가총액','상장주식수','외국인비율','거래량','PER','ROE','토론실'])

In [2]:
#구글 이미지 다운로드 
import requests

url = "https://cdn.travie.com/news/photo/first/201710/img_19975_1.jpg"

response =requests.get(url)

print(response.status_code)

200


In [4]:
if response. status_code == 200:
    # response 에서 받아온 이미지(binary)를 조회
    # result=response.content
    # print(type(result))
    with open ('image.jpg', "wb") as fw:
        fw.write(response.content)

In [ ]:
# 여러개를 한꺼번에 저장하는것 
#article-view-content-div > div > div  img
# 1. 사이트를 불러옴
# 2. 이미지 url 파서를 함
# 3. url 을 리스트에 받음
# 4. 리스트에 for 문으로 url 을 빼서 with open image를 해서 전부 저장 하면 됨
# 5. 그러면 리스트에 알아서 뽑아가짐
# 6. 주말에 해보기